In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [48]:
states = vq.get_initial_state()
events = vq.get_event_table()
player_alive_dict = {item: True for item in states["Player ID"].unique()}

current_round_alive_dict = player_alive_dict.copy()
chosen_round = 7
current_round_states = states[states['Round'] == chosen_round]
current_round_events = events[events['Round'] == chosen_round]

attacking_players = tuple(current_round_states[current_round_states['Starting Side'] == 'attack']['Player ID'].unique()) if chosen_round <= 12 else tuple(current_round_states[current_round_states['Starting Side'] == 'defense']['Player ID'].unique())
defending_players = tuple(current_round_states[current_round_states['Starting Side'] == 'defense']['Player ID'].unique()) if chosen_round <= 12 else tuple(current_round_states[current_round_states['Starting Side'] == 'attack']['Player ID'].unique())

current_gamestate_list = []

first_state = vq.get_current_gamestate(current_round_states, current_round_alive_dict, attacking_players, defending_players)
current_gamestate_list.append(first_state)
for event in current_round_events.iterrows():
    event_type = event[1]["EventType"]
    victim_id = event[1]["VictimID"]
    if event_type == "kill":
        current_round_alive_dict[victim_id] = False
    elif event_type == "revival":
        current_round_alive_dict[victim_id] = True
    gs = vq.get_current_gamestate(current_round_states, current_round_alive_dict, attacking_players, defending_players)
    current_gamestate_list.append(gs)
    player_data = current_round_states[current_round_states['Player ID'] == victim_id]

dummy_gamestate_list = []
for gamestate in current_gamestate_list:
    # Append "ATK" to the beginning of the keys of the attack gamestate
    atk_gamestate = {f"ATK {key}".replace(" ", "_"): value for key, value in gamestate["atk_data"].items()}
    def_gamestate = {f"DEF {key}".replace(" ", "_"): value for key, value in gamestate["def_data"].items()}
    merged_gamestate = {**atk_gamestate, **def_gamestate}
    dummy_gamestate_list.append(merged_gamestate)
gamestate_df = pd.DataFrame(dummy_gamestate_list)
timestamps = list(current_round_events["Round_time_millis"])
timestamps.insert(0, 0)
gamestate_df.insert(0, "Timestamp", timestamps)
event_type_column = list(current_round_events["EventType"])
event_type_column.insert(0, "start")
gamestate_df.insert(1, "EventType", event_type_column)
gamestate_df

,Timestamp,EventType,ATK_Loadout_Value,ATK_Shield,ATK_Weapon_Price,ATK_Initiator,ATK_Duelist,ATK_Controller,ATK_Sentinel,DEF_Loadout_Value,DEF_Shield,DEF_Weapon_Price,DEF_Initiator,DEF_Duelist,DEF_Controller,DEF_Sentinel
0,0,start,23200,250,14500,1,2,1,1,24950,225,16600,2,1,1,1
1,7526,kill,18600,200,11600,0,2,1,1,24950,225,16600,2,1,1,1
2,12106,revival,23200,250,14500,1,2,1,1,24950,225,16600,2,1,1,1
3,16686,kill,18700,200,11600,1,2,0,1,24950,225,16600,2,1,1,1
4,17968,kill,18700,200,11600,1,2,0,1,19000,200,11600,2,0,1,1
5,79775,kill,18700,200,11600,1,2,0,1,13600,150,8700,1,0,1,1
6,89060,plant,18700,200,11600,1,2,0,1,13600,150,8700,1,0,1,1
7,91326,revival,18700,200,11600,1,2,0,1,19000,200,11600,2,0,1,1
8,93591,kill,14100,150,8700,1,1,0,1,19000,200,11600,2,0,1,1
9,94639,kill,9300,100,5800,1,0,0,1,19000,200,11600,2,0,1,1


In [4]:
vq.aggregate_gamestate(5)

,Timestamp,EventType,ATK_Loadout_Value,ATK_Shield,ATK_Weapon_Price,ATK_Initiator,ATK_Duelist,ATK_Controller,ATK_Sentinel,DEF_Loadout_Value,DEF_Shield,DEF_Weapon_Price,DEF_Initiator,DEF_Duelist,DEF_Controller,DEF_Sentinel
0,0,start,24000,250,14500,1,2,1,1,23250,200,16600,2,1,1,1
1,72891,kill,24000,250,14500,1,2,1,1,18550,150,13700,2,1,1,0
2,73216,kill,18500,200,11600,1,2,1,0,18550,150,13700,2,1,1,0
3,80084,kill,13900,150,8700,0,2,1,0,18550,150,13700,2,1,1,0
4,80351,kill,13900,150,8700,0,2,1,0,12900,125,8700,2,0,1,0
5,86246,kill,13900,150,8700,0,2,1,0,8300,75,5800,1,0,1,0
6,88979,kill,13900,150,8700,0,2,1,0,3700,25,2900,0,0,1,0
7,92160,plant,13900,150,8700,0,2,1,0,3700,25,2900,0,0,1,0
8,95954,kill,13900,150,8700,0,2,1,0,0,0,0,0,0,0,0


### AAA